In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)

from datetime import datetime
from langchain.llms import OpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI


f = open('api_key.txt')
api_key = f.read()

In [22]:
class HistoryQuiz():

    def create_history_question(self, topic):
        '''
        This method should output a historical question about the topic that has a data as the question
        For example:


            "On what date did World War 2 end?"

        '''

        # SYSTEM
        system_template = "You write a single quiz questions about {topic}. You only return the qui question."
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

        # HUMAN
        human_template = "{question_request}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        # Compile ChatTemplate
        chat_prompt = ChatPromptTemplate.from_messages(
            [
                system_message_prompt,
                human_message_prompt
            ]
        )

        # Insert topic
        q = "Give me a quiz question where the correct answer is specific date."
        request = chat_prompt.format_prompt(topic=topic, question_request=q).to_messages()

        # Request

        chat = ChatOpenAI(openai_api_key=api_key)

        result = chat(request)

        return result.content


    def get_AI_answer(self, question):

        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!

        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''

        # Datetime Parser
        output_parser = DatetimeOutputParser()
        # System Template
        system_template = "You answer quiz questions with just a date. like datetime.datetime object"
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        # Human template --> format_instructions
        human_template = """Answer the user's question:

        {question}


        {format_instructions}"""
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        # Compile ChatPromptTemplate
        chat_prompt = ChatPromptTemplate.from_messages(
            [
                system_message_prompt,
                human_message_prompt
            ]
        )
        # Insert question and get_format_instructions
        format_instructions = output_parser.get_format_instructions()
        request = chat_prompt.format_prompt(
            question=question,
            format_instructions=format_instructions
        ).to_messages()
        # chatbot result --> format datetime
        chat = ChatOpenAI(openai_api_key=api_key)
        result = chat(request)

        correct_datetime = output_parser.parse(result.content)


        return correct_datetime


    def get_user_answer(self, question):
        '''
        This method should grab a user answer and convert it to datetime. It should correct an answer.
        You can just use input() for this.
        '''
        print(question)
        print('\n')
        year = int(input("Enter the year: "))
        month = int(input("Enter the month (1-12): "))
        day = int(input("Enter the day (1-31): "))
        user_datetime = datetime(year, month, day)


        return user_datetime


    def check_user_answer(self, user_answer, ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''

        # print or return difference between the answer here!
        difference = user_answer - ai_answer
        formatted_difference = str(difference)
        print(f"The difference between answer and your guess: {formatted_difference}")

In [23]:
quiz_bot = HistoryQuiz()

In [24]:
question = quiz_bot.create_history_question(topic='Neil Armstrong set foot moon date')

In [25]:
question

'On what date did Neil Armstrong set foot on the moon?'

In [26]:
ai_answer = quiz_bot.get_AI_answer(question)

In [27]:
ai_answer

datetime.datetime(1969, 7, 20, 20, 17, 40)

In [28]:
user_answer = quiz_bot.get_user_answer(question)

On what date did Neil Armstrong set foot on the moon?




In [29]:
user_answer

datetime.datetime(1969, 1, 2, 0, 0)

In [30]:
quiz_bot.check_user_answer(user_answer, ai_answer)

The difference between answer and your guess: -200 days, 3:42:20
